In [1]:
import torchreid
model_name = 'resnet50'
dataset_name = 'market1501'
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources=dataset_name,
    targets=dataset_name,
    height=256,
    width=128,
    batch_size_train=128,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [2]:
model = torchreid.models.build_model(
    name=model_name,
    num_classes=datamanager.num_train_pids, #2000 for Tango
    loss="triplet",
    pretrained=True,
    pooling='gem'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [5]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [3]:
engine = torchreid.engine.ImageTripletEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [4]:
engine.run(
    save_dir=f"log/{model_name}_{dataset_name}",
    max_epoch=60,
    eval_freq=5,
    print_freq=5000,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training


100%|██████████| 101/101 [00:16<00:00,  5.98it/s]


##### Evaluating market1501 (source) #####
Extracting features from query set ...


100%|██████████| 34/34 [00:01<00:00, 28.30it/s]


Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...


...

=> Final test
##### Evaluating market1501 (source) #####
Extracting features from query set ...


100%|██████████| 34/34 [00:01<00:00, 27.39it/s]


Done, obtained 3368-by-2048 matrix
Extracting features from gallery set ...


100%|██████████| 160/160 [00:05<00:00, 29.79it/s]


Done, obtained 15913-by-2048 matrix
Speed: 0.0018 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
[   0  455    0 ... 1026  668  723] <class 'numpy.ndarray'> False
** Results **
mAP: 72.2%
CMC curve
Rank-1  : 88.0%
Rank-5  : 95.0%
Rank-10 : 96.8%
Rank-20 : 97.9%
Checkpoint saved to "log/resnet50_market1501/model/model.pth.tar-60"
Elapsed 0:25:08
